# Prompt Engineering Analysis with Anthropic's Claude

This notebook analyzes how different prompt engineering techniques affect Claude's output distributions, response characteristics, and behavior.

## Topics Covered:
1. Setting up the Anthropic API
2. Comparing different prompt engineering techniques
3. Analyzing temperature effects on output distribution
4. Visualizing output diversity and characteristics
5. Statistical analysis of prompt effectiveness

## Setup

In [ ]:
import anthropic
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Any
from collections import Counter
import time

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

# Import our analyzer
from prompt_engineering_analyzer import PromptEngineeringAnalyzer

print("Setup complete!")

: 

## Configure API Key

Make sure you have your Anthropic API key set as an environment variable:
```bash
export ANTHROPIC_API_KEY='your-api-key-here'
```

Or set it directly in this notebook (not recommended for production):

In [ ]:
# Option 1: Use environment variable (recommended)
api_key = os.environ.get("ANTHROPIC_API_KEY")

# Option 2: Set directly (uncomment and use with caution)
# api_key = "your-api-key-here"

if not api_key:
    print("⚠️ WARNING: ANTHROPIC_API_KEY not found!")
    print("Please set your API key before proceeding.")
else:
    print("✓ API key configured")

## Initialize Analyzer

In [ ]:
# Initialize the analyzer with your preferred model
analyzer = PromptEngineeringAnalyzer(
    api_key=api_key,
    model="claude-3-5-sonnet-20241022"  # or "claude-3-opus-20240229", etc.
)

print(f"Analyzer initialized with model: {analyzer.model}")

## 1. Basic Prompt Testing

Let's start with a simple example to see how Claude responds to a basic prompt.

In [ ]:
# Test a simple prompt
test_prompt = "Explain what a neural network is in one sentence."

response = analyzer.get_response(test_prompt, temperature=0.7)

print("Prompt:", test_prompt)
print("\nResponse:", response['response_text'])
print("\nMetadata:")
print(f"  - Input tokens: {response['input_tokens']}")
print(f"  - Output tokens: {response['output_tokens']}")
print(f"  - Response length: {response['response_length']} characters")
print(f"  - Stop reason: {response['stop_reason']}")

## 2. Comparing Prompt Engineering Techniques

Now let's compare different prompt engineering approaches on the same question.

In [ ]:
# Define your question
base_question = "What are the three most important factors in training a deep learning model?"

# Define different prompt engineering variants
prompt_variants = {
    "baseline": {
        "prompt": base_question,
        "system": ""
    },
    
    "with_system_prompt": {
        "prompt": base_question,
        "system": "You are an expert deep learning researcher with 10 years of experience."
    },
    
    "chain_of_thought": {
        "prompt": f"{base_question}\n\nLet's think through this step by step:",
        "system": ""
    },
    
    "structured": {
        "prompt": f"{base_question}\n\nProvide your answer in this format:\n1. [Factor]: [Explanation]\n2. [Factor]: [Explanation]\n3. [Factor]: [Explanation]",
        "system": ""
    },
    
    "few_shot": {
        "prompt": f"""Here's an example of explaining key factors:

Q: What are the three most important factors in building a web application?
A: The three most important factors are:
1. Security - Protecting user data and preventing vulnerabilities
2. Performance - Ensuring fast load times and responsive interactions
3. User Experience - Creating an intuitive and accessible interface

Q: {base_question}
A:""",
        "system": ""
    },
    
    "role_playing": {
        "prompt": base_question,
        "system": "You are Yann LeCun, discussing deep learning with a colleague. Be concise but insightful."
    }
}

print("Prompt variants defined:")
for name in prompt_variants.keys():
    print(f"  - {name}")

### Run the comparison (this will take a few minutes)

In [ ]:
# Compare all variants
print("Running comparison... This may take a few minutes.\n")

df_variants = analyzer.compare_prompts(
    base_question=base_question,
    prompt_variants=prompt_variants,
    temperature=0.7,
    num_samples=5  # 5 samples per variant
)

print("\n✓ Comparison complete!")
print(f"\nTotal responses collected: {len(df_variants)}")

### View sample responses

In [ ]:
# Display one sample from each variant
for variant in df_variants['variant'].unique():
    sample = df_variants[df_variants['variant'] == variant].iloc[0]
    print("="*80)
    print(f"Variant: {variant}")
    print("="*80)
    print(f"Response:\n{sample['response_text']}")
    print(f"\nTokens: {sample['output_tokens']} | Length: {sample['response_length']} chars\n")

### Analyze output diversity

In [ ]:
diversity_metrics = analyzer.analyze_output_diversity(df_variants)

# Convert to DataFrame for better display
diversity_df = pd.DataFrame(diversity_metrics).T
diversity_df = diversity_df.round(3)

print("Output Diversity Metrics:\n")
print(diversity_df)

# Highlight key insights
print("\n" + "="*80)
print("KEY INSIGHTS:")
print("="*80)

most_diverse = diversity_df['uniqueness_ratio'].idxmax()
least_diverse = diversity_df['uniqueness_ratio'].idxmin()

print(f"\nMost diverse responses: {most_diverse} (ratio: {diversity_df.loc[most_diverse, 'uniqueness_ratio']:.3f})")
print(f"Least diverse responses: {least_diverse} (ratio: {diversity_df.loc[least_diverse, 'uniqueness_ratio']:.3f})")

longest = diversity_df['avg_response_length'].idxmax()
shortest = diversity_df['avg_response_length'].idxmin()

print(f"\nLongest responses: {longest} ({diversity_df.loc[longest, 'avg_response_length']:.0f} chars avg)")
print(f"Shortest responses: {shortest} ({diversity_df.loc[shortest, 'avg_response_length']:.0f} chars avg)")

### Visualize the comparison

In [ ]:
analyzer.visualize_prompt_comparison(df_variants, save_path="prompt_comparison.png")

## 3. Temperature Analysis

Now let's see how temperature affects the output distribution for a single prompt.

In [ ]:
# Define a prompt for temperature analysis
temp_test_prompt = "Name three creative uses for artificial intelligence in healthcare."

print("Running temperature analysis...\n")
print(f"Prompt: {temp_test_prompt}")
print(f"Temperatures to test: [0.0, 0.3, 0.7, 1.0, 1.5]\n")

df_temperature = analyzer.analyze_temperature_effects(
    prompt=temp_test_prompt,
    system="You are a creative AI assistant.",
    temperatures=[0.0, 0.3, 0.7, 1.0, 1.5],
    num_samples=10
)

print("\n✓ Temperature analysis complete!")

### View responses at different temperatures

In [ ]:
# Show one example from each temperature
for temp in sorted(df_temperature['temperature'].unique()):
    sample = df_temperature[df_temperature['temperature'] == temp].iloc[0]
    print("="*80)
    print(f"Temperature: {temp}")
    print("="*80)
    print(sample['response_text'])
    print()

### Temperature statistics

In [ ]:
# Calculate statistics by temperature
temp_stats = df_temperature.groupby('temperature').agg({
    'response_length': ['mean', 'std', 'min', 'max'],
    'word_count': ['mean', 'std'],
    'output_tokens': ['mean', 'std']
}).round(2)

print("Statistics by Temperature:\n")
print(temp_stats)

# Calculate uniqueness ratio by temperature
print("\n" + "="*80)
print("Response Uniqueness by Temperature:")
print("="*80)

for temp in sorted(df_temperature['temperature'].unique()):
    responses = df_temperature[df_temperature['temperature'] == temp]['response_text'].tolist()
    unique_ratio = len(set(responses)) / len(responses)
    print(f"Temperature {temp}: {unique_ratio:.2%} unique responses ({len(set(responses))}/{len(responses)})")

### Visualize temperature effects

In [ ]:
analyzer.visualize_temperature_effects(df_temperature, save_path="temperature_effects.png")

## 4. Custom Analysis: Response Patterns

Let's do some custom analysis on common patterns in the responses.

In [ ]:
def analyze_response_patterns(df, text_column='response_text'):
    """
    Analyze common patterns in responses.
    """
    patterns = {
        'starts_with_number': 0,
        'contains_list': 0,
        'contains_bullet': 0,
        'starts_with_capital': 0,
        'contains_colon': 0,
    }
    
    for text in df[text_column]:
        if text[0].isdigit():
            patterns['starts_with_number'] += 1
        if any(text.startswith(f"{i}.") or f"\n{i}." in text for i in range(1, 10)):
            patterns['contains_list'] += 1
        if '•' in text or '- ' in text or '* ' in text:
            patterns['contains_bullet'] += 1
        if text[0].isupper():
            patterns['starts_with_capital'] += 1
        if ':' in text:
            patterns['contains_colon'] += 1
    
    # Convert to percentages
    total = len(df)
    for key in patterns:
        patterns[key] = (patterns[key] / total) * 100
    
    return patterns

# Analyze patterns by variant
print("Response Patterns by Variant:\n")
print("=" * 80)

for variant in df_variants['variant'].unique():
    variant_df = df_variants[df_variants['variant'] == variant]
    patterns = analyze_response_patterns(variant_df)
    
    print(f"\n{variant}:")
    for pattern, percentage in patterns.items():
        print(f"  {pattern}: {percentage:.1f}%")

## 5. Word Frequency Analysis

Analyze which words appear most frequently in different prompt variants.

In [ ]:
from collections import Counter
import re

def get_top_words(df, variant, n=20, min_length=4):
    """
    Get top N words for a specific variant.
    """
    # Get all responses for this variant
    texts = df[df['variant'] == variant]['response_text'].tolist()
    
    # Combine all text
    combined_text = ' '.join(texts).lower()
    
    # Extract words (alphanumeric only)
    words = re.findall(r'\b[a-z]+\b', combined_text)
    
    # Filter by length and remove common stop words
    stop_words = {'the', 'this', 'that', 'with', 'from', 'have', 'they', 'will', 'your', 
                  'more', 'about', 'which', 'their', 'there', 'than', 'them', 'these',
                  'been', 'were', 'when', 'where', 'also', 'can', 'are', 'and', 'for'}
    
    words = [w for w in words if len(w) >= min_length and w not in stop_words]
    
    # Count and return top N
    return Counter(words).most_common(n)

# Analyze top words for each variant
print("Top 15 Words by Variant:\n")
print("=" * 80)

for variant in df_variants['variant'].unique():
    print(f"\n{variant}:")
    top_words = get_top_words(df_variants, variant, n=15)
    
    for word, count in top_words:
        print(f"  {word}: {count}")

## 6. Visualize Word Frequency Comparison

In [ ]:
# Create a comparison of top words across variants
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for idx, variant in enumerate(df_variants['variant'].unique()):
    if idx >= len(axes):
        break
        
    top_words = get_top_words(df_variants, variant, n=10)
    words, counts = zip(*top_words)
    
    axes[idx].barh(words, counts, color=sns.color_palette("husl", 10))
    axes[idx].set_xlabel('Frequency')
    axes[idx].set_title(f'Top Words: {variant}')
    axes[idx].invert_yaxis()

# Hide any unused subplots
for idx in range(len(df_variants['variant'].unique()), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('word_frequency_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Save Results

In [ ]:
# Save all results
analyzer.save_results(df_variants, "prompt_variants_results")
analyzer.save_results(df_temperature, "temperature_analysis_results")

print("Results saved!")
print("\nGenerated files:")
print("  - prompt_variants_results.csv")
print("  - prompt_variants_results.json")
print("  - temperature_analysis_results.csv")
print("  - temperature_analysis_results.json")
print("  - prompt_comparison.png")
print("  - temperature_effects.png")
print("  - word_frequency_comparison.png")

## 8. Your Custom Experiments

Use this section to run your own experiments!

In [ ]:
# Define your own question and variants here

my_question = "Your question here"

my_variants = {
    "variant_1": {
        "prompt": my_question,
        "system": ""
    },
    # Add more variants...
}

# Run your experiment
# my_results = analyzer.compare_prompts(my_question, my_variants, temperature=0.7, num_samples=5)
# analyzer.visualize_prompt_comparison(my_results)

## Summary

This notebook demonstrated:

1. **Prompt Engineering Techniques**: How different prompting strategies (zero-shot, few-shot, chain-of-thought, etc.) affect outputs
2. **Temperature Effects**: How temperature influences output diversity and creativity
3. **Output Analysis**: Measuring response length, diversity, and patterns
4. **Word Frequency**: Understanding vocabulary usage across different prompts

Key findings you might observe:
- Lower temperatures (0.0-0.3) produce more consistent, deterministic outputs
- Higher temperatures (1.0-1.5) increase diversity but may reduce coherence
- Structured prompts tend to produce more consistent formatting
- System prompts can significantly influence tone and style
- Few-shot examples guide the model toward specific response patterns